In [ ]:
from traitlets import Unicode, Bool, validate, TraitError, observe
from ipywidgets import DOMWidget, register
import time
import threading

In [ ]:
%%javascript

require.undef('email_widget');

// sleep time expects milliseconds
function sleep (time) {
    return new Promise((resolve) => setTimeout(resolve, time));
}

define('email_widget', ["@jupyter-widgets/base"], function(widgets) {
    var EmailView = widgets.DOMWidgetView.extend({
        render: function() {
            this.email_input = document.createElement('input');
            this.email_input.type = 'email';
            this.email_input.value = this.model.get('pipe_js2py');
            this.el.appendChild(this.email_input);
            
            // Python -> JavaScript update
            this.model.on('change:pipe_js2py', this.on_pipe_js2py, this);
            
            // JavaScript -> Python update
            // this.email_input.onchange = this.input_changed.bind(this);
            sleep(1000).then(() => {
                this.model.set('pipe_js2py', "js2py message: A");
                this.touch();
            }).then(() => sleep(1000)).then(() => {
                this.model.set('pipe_js2py', "js2py message: B");
                this.touch();
            }).then(() => sleep(1000)).then(() => {
                this.model.set('pipe_js2py', "js2py message: C");
                this.touch();
            });
        },
        
        on_pipe_js2py: function() {
            this.email_input.value = this.model.get('pipe_js2py');
        },
        
        input_changed: function() {
            this.model.set('pipe_js2py', this.email_input.value);
            // Use touch(), recommended by
            // https://github.com/jupyter-widgets/ipywidgets/issues/1783#issuecomment-340312257
            this.touch()
            // this.model.save_changes();
        },
    });
    
    return {
        EmailView: EmailView
    }
});

In [ ]:
def change_value_five_times(widget):
    def func():
        for i in range(5):
            widget.pipe_js2py = f"update_{i}@gmail.com"
            time.sleep(1)
    return func

@register
class Email(DOMWidget):
    _view_name = Unicode('EmailView').tag(sync=True)
    _view_module = Unicode('email_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    
    # Attributes
    pipe_py2js = Unicode("Default py->js pipe value.", help="Python to JS pipe.").tag(sync=True)
    pipe_js2py = Unicode("Default js->py pipe value.", help="JS to Python pipe.").tag(sync=True)

    @observe('pipe_js2py')
    def _observe_pipe_js2py(self, change):
        print(f"pipe_js2py changed: {change['old']} -> {change['new']}")

In [ ]:
email = Email(pipe_py2js='init_py2js', pipe_js2py='init_js2py')
email

In [ ]:
# thread = threading.Thread(target=change_value_five_times(email))
# thread.start()